In [1]:
import torch
import torch.nn as nn
import torch.optim as opt
import torch.nn.functional as fun
import numpy as np
import pandas as pd

# Load do dataset

In [2]:
from lib.data import load_dataset

rs = 2

x_train, y_train, x_valid, y_valid = load_dataset('dts/dts_all/train.csv',rs=rs)
x_test = load_dataset('dts/dts_all/test.csv',test_only=True,rs=rs)

# Normalização

In [3]:
from sklearn.preprocessing import StandardScaler
from my_nn import dts

sc = StandardScaler()
sc.fit(x_train.values)

x_train = sc.transform(x_train.values)
x_valid = sc.transform(x_valid.values)
x_test = sc.transform(x_test.values)

# Feature Selection

In [4]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

selec_mod = SelectFromModel(estimator=LogisticRegression(random_state=rs, max_iter=2000)).fit(x_train, y_train)

x_train = selec_mod.transform(x_train)
x_valid = selec_mod.transform(x_valid)
x_test = selec_mod.transform(x_test)

# Dataloaders

In [5]:
from torch.utils.data import DataLoader
train_dts = dts(x_train,y_train)
valid_dts = dts(x_valid,y_valid)

training_loader = DataLoader(train_dts, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dts, batch_size=32, shuffle=True)

In [6]:
from my_nn import nnModel

model = nnModel(inFeatures = x_train.shape[1], random_state=2)

losses, accs = model.fit(
    trainloader=training_loader,
    validloader=valid_loader,
    learningRate=0.02,
    momentum=0.9,
    gamma=0.97,
    numEpochs=31,
    verbose=False
)

In [7]:
print(losses)
print(accs)

from matplotlib import pyplot as plt
plt.figure()
plt.plot(losses)
plt.figure()
plt.plot(accs)

[312.0350165888667, 139.13804518431425, 94.4781698101433, 70.1708695476409, 56.0463121564826, 43.99297285153443, 33.11418988841615, 34.19973933559231, 24.13231306803209, 19.16263302202242, 18.346706016865937, 14.356849189477089, 7.865356482076095, 9.786641216341366, 8.716525891660467, 3.645111675194812, 1.040884500453938, 0.7917869780638682, 0.14220350017346917, 0.10575282124547947, 0.08527706824336434, 0.07449293334402718, 0.06669637882606771, 0.06018847404930483, 0.05565393308912636, 0.05123707126556809, 0.04836656113952387, 0.045397581565113754, 0.04274125557977415, 0.04064049559166527, 0.038654889739459186]
[97.25, 97.35, 98.1375, 98.325, 98.2875, 98.2625]


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

yvalid = model.predict(torch.tensor(x_valid, dtype=torch.float32))

print("Acc: {:.3f}".format(accuracy_score(y_true=y_valid, y_pred=yvalid)))
print("ROC: {:.3f}".format(roc_auc_score(y_valid, yvalid)))
print("Acc: {:.3f}".format(precision_score(y_true=y_valid, y_pred=yvalid)))
print("Acc: {:.3f}".format(recall_score(y_true=y_valid, y_pred=yvalid)))

Acc: 0.983
ROC: 0.982
Acc: 0.986
Acc: 0.978


In [ ]:
ypred = model.predict(torch.tensor(x_test, dtype=torch.float32))
import pandas as pd
result = pd.DataFrame(data={
        "Id": range(len(ypred)),
        "Category": ypred.astype(int)
    }, index=None)

result.to_csv("result.csv", index=None)